## Preparing Data

In [38]:
##todo 
# 1 分析数据
# 2 复制模型
# 3 插入其中
# 4 运行
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from  torchtext.data import Field
import torchtext
tok=lambda x: x.split()
TEXT = Field(tokenize=tok,eos_token='<eos>',init_token='<sos>',include_lengths=True)
LABEL = Field(tokenize=tok,eos_token='<eos>',init_token='<sos>',include_lengths=True)

In [59]:
import random
import math
import time

In [60]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
path='../data/train/dbn.in.txt'
trg_path = '../data/train/dbn.out.txt'

In [4]:
lists=open(path,'r',encoding='utf-8').readlines()
trg_lists = open(trg_path,'r',encoding='utf-8').readlines()

In [5]:
lists[:3]
trg_lists[:3]

['. ) ) ) ) ) ) ) ) . . . . ( ( . ( ( ( ( ( . . . ( ( . . ( ( ( ( ( ( . . . . . . ) ) . . ) ) ) ) . . ) ) . . . . ) ) ) ) ) . . ) ) ) ) ) . ) ) . . . . ) . ( ( . . . . . ( ( . . . . ) ) . . . . . ) ) . ( . . ( ( . ( ( ( ( ( ( ( ( ( ( ( \n',
 '. ) ) ) ) ) . . . . ( ( ( ( ( . . . . . . . . . . . . ) ) ) ) ) . . ) ) . . . ) ) ) ) ) ) ) ) ) ) ) . . . . . . . . . . . ) ) ) ) ) ) . . . . . ) ) ) ) ) . ) ) ) ) . ) ) ) ) . . ) ) ) ) . . . . . . . . ( ( ( ( . . . ( ( ( ( . . . . . . . . . . ( ( ( ( ( ( . ( ( ( . . ( ( ( ( ( ( ( ( ( ( ( . ( ( ( ( ( ( . . . . ( ( . . ( ( ( ( \n',
 '. ) ) ) ) ) ) ) . . . . ( ( . ( ( ( ( ( . . . ( ( . . ( ( ( ( ( ( . . . . . . . ) ) . . ) ) ) ) . . ) ) . . . . ) ) ) ) ) . . ) ) ) ) ) . ) ( . . ( . ( ( . . . ( ( . . . . ) ) . . . . . ) ) . ) . ) ( . ( ( ( . ( ( ( ( ( ( ( . . \n']

In [6]:
s=LABEL.preprocess(trg_lists[0])
s

['.',
 ')',
 ')',
 ')',
 ')',
 ')',
 ')',
 ')',
 ')',
 '.',
 '.',
 '.',
 '.',
 '(',
 '(',
 '.',
 '(',
 '(',
 '(',
 '(',
 '(',
 '.',
 '.',
 '.',
 '(',
 '(',
 '.',
 '.',
 '(',
 '(',
 '(',
 '(',
 '(',
 '(',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 ')',
 ')',
 '.',
 '.',
 ')',
 ')',
 ')',
 ')',
 '.',
 '.',
 ')',
 ')',
 '.',
 '.',
 '.',
 '.',
 ')',
 ')',
 ')',
 ')',
 ')',
 '.',
 '.',
 ')',
 ')',
 ')',
 ')',
 ')',
 '.',
 ')',
 ')',
 '.',
 '.',
 '.',
 '.',
 ')',
 '.',
 '(',
 '(',
 '.',
 '.',
 '.',
 '.',
 '.',
 '(',
 '(',
 '.',
 '.',
 '.',
 '.',
 ')',
 ')',
 '.',
 '.',
 '.',
 '.',
 '.',
 ')',
 ')',
 '.',
 '(',
 '.',
 '.',
 '(',
 '(',
 '.',
 '(',
 '(',
 '(',
 '(',
 '(',
 '(',
 '(',
 '(',
 '(',
 '(',
 '(']

In [7]:
examples=[]

In [8]:
for x,t in zip(lists,trg_lists):
    e=torchtext.data.Example()
    setattr(e,'text',TEXT.preprocess(x))
    setattr(e,'label',LABEL.preprocess(t))
    examples.append(e)

In [9]:
examples[0]

In [10]:
from collections import Counter

In [11]:
TEXT.__dict__.keys()

dict_keys(['sequential', 'use_vocab', 'init_token', 'eos_token', 'unk_token', 'fix_length', 'dtype', 'preprocessing', 'postprocessing', 'lower', 'tokenizer_args', 'tokenize', 'include_lengths', 'batch_first', 'pad_token', 'pad_first', 'truncate_first', 'stop_words', 'is_target'])

In [12]:
c_text=Counter()
c_trg =Counter()
for e in examples:
    c_text.update(e.text)
    c_trg.update(e.label)
c_text

Counter({'A': 527195,
         'G': 543033,
         'C': 490090,
         'U': 506882,
         'N': 190,
         'K': 8,
         'S': 4,
         'M': 8,
         'W': 10,
         'R': 12,
         'Y': 15,
         'B': 1,
         'V': 1})

In [13]:
c_trg

Counter({'.': 1091275, ')': 488259, '(': 487915})

In [14]:
specials=[TEXT.unk_token,TEXT.pad_token,TEXT.init_token,TEXT.eos_token]   # 请把unk_token 放第一位，因为源码会直接把unk词映射到0
trg_specials=[TEXT.unk_token,TEXT.pad_token,TEXT.init_token,TEXT.eos_token]

In [16]:
TEXT.vocab=TEXT.vocab_cls(c_text,specials=specials,min_freq=1000)
LABEL.vocab=LABEL.vocab_cls(c_trg,specials=trg_specials)


In [18]:
len(LABEL.vocab)

7

In [19]:
LABEL.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000001479E14D828>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             '.': 4,
             ')': 5,
             '(': 6})

In [26]:
dataset=torchtext.data.Dataset(examples,{'text':TEXT,'label':LABEL})

In [27]:
dataset.__dict__.keys()

dict_keys(['examples', 'fields'])

In [28]:
len(dataset)

14912

In [29]:
ite=torchtext.data.BucketIterator(dataset,batch_size=32,sort_key=lambda x:len(x.text),sort_within_batch=True,device=-1)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [30]:
Iter=iter(ite)

In [31]:
next(Iter)


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 117x32]', '[torch.LongTensor of size 32]')
	[.label]:('[torch.LongTensor of size 117x32]', '[torch.LongTensor of size 32]')

In [32]:
batch=next(Iter)

In [33]:
batch.__dict__.keys()

dict_keys(['batch_size', 'dataset', 'fields', 'input_fields', 'target_fields', 'text', 'label'])

In [36]:
batch.text[0]
batch.label[0]

tensor([[2, 2, 2,  ..., 2, 2, 2],
        [4, 4, 4,  ..., 4, 4, 4],
        [5, 6, 5,  ..., 4, 4, 4],
        ...,
        [4, 4, 4,  ..., 3, 3, 3],
        [4, 4, 4,  ..., 1, 1, 1],
        [3, 3, 3,  ..., 1, 1, 1]])

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Building the Seq2Seq Model

In [39]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [40]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #sent len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [41]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

## Train Model

In [49]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(LABEL.vocab)
ENC_EMB_DIM = 4
DEC_EMB_DIM = 4
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [50]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8, 4)
    (rnn): LSTM(4, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7, 4)
    (rnn): LSTM(4, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=7, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [51]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,327,875 trainable parameters


In [52]:
optimizer = optim.Adam(model.parameters())

In [54]:
PAD_IDX = TEXT.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [62]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.text
        trg = batch.label
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [63]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.text
            trg = batch.label

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [57]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [61]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, ite, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

NameError: name 'train_iterator' is not defined